In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from io import BytesIO
from importlib import reload
from lib import run

# Refactor

lib.py was refactored ... 

In [ ]:
ADDRESS = ('91.204.99.178', 8333)

In [ ]:
from network import *

peer = PeerConnection(*ADDRESS)

peer.handshake()

Our goal is initial block download on the first 10,000 bitcoin blocks. Bitcoin does initial block download by first downloading "block headers" -- which are blocks without the transactions -- and validating their hashes form and chain and that they satisfy bitcoin's proof of work requirements. Once the longest valid chain of block headers has been discovered and validated, then the bitcoin software proceeds to download and the full blocks -- block headers plus all the transactions. This takes a lot longer, but can be downloaded in parallel from multiple peers, which speeds up the process. 

First of all, we need to figure out how to download block headers. The bitcoin wiki tells us how: behold the We need to send a getheaders message to request the [`getheaders`](https://en.bitcoin.it/wiki/Protocol_documentation#getheaders) network message.

# `getheaders`

[network.py](./network.py) contains a stub for a `GetHeadersMessage` class. Let's fill it out so that we can send a `getheaders` message in the cell below and get a first batch of headers from a bitcoin peer.

In [ ]:
import network as network

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

Now that we have a class abstracting the `getheaders` message, let's put it to use. Use this message to get the next block after genesis ...

In [ ]:
from network import HeadersMessage as HeadersMessage

# connect to peer
peer = PeerConnection(*ADDRESS, logging=True)
peer.handshake()

# construct the message
genesis_hash_hex = '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'
genesis_hash_bytes = bytes.fromhex(genesis_hash_hex)
getheaders = GetHeadersMessage(start_block=genesis_hash_bytes)

# send the message
peer.send(getheaders)

# see what we get back
while True:
    peer.read()

In [ ]:
import network as network

reload(network)
run(network.HeadersMessageTest("test_parse"))

In [ ]:
from network import HeadersMessageInitial as HeadersMessage

# construct the message
genesis_hash_hex = '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'
genesis_hash_bytes = bytes.fromhex(genesis_hash_hex)
getheaders = GetHeadersMessage(start_block=genesis_hash_bytes)

# send the message
peer.send(getheaders)

# wait for the "headers" response
headers_payload = peer.wait_for(HeadersMessage)
headers_payload

Now that we need to learn to interpret the `headers` response we are getting. Let's implement `HeadersMessage.parse` to make this happen. If we look at the protocol docs we see that the payload of the `headers` message contains, as we should expect, block header objects. In order to implement `HeadersMessage.parse` we first need to implement implement a `BlockHeader` class with a `.parse` method which `HeadersMessage.parse` can call.

### `BlockHeader`

block.py has the stub of a `BlockHeader` class. Fill out the `BlockHeader.parse` method to get the test below to pass.

In [ ]:
import network as network

reload(network)
run(network.HeadersMessageTest("test_parse"))

Now that we have a `HeadersMessage` that passes the tests, let's use it to interpret the `headers` payload bytes we received earlier:

In [ ]:
from network import HeadersMessage

headers_msg = HeadersMessage.parse(BytesIO(headers_payload))
headers_msg.headers

In [ ]:
headers_msg.headers[0]

In [ ]:
headers_msg.headers[0].prev_block

Very good! Now let's try to call this repeatedly and build up a chain of block headers:

In [ ]:
from block import RAW_GENESIS_BLOCK

# Create a list of headers with genesis block filled in
GENESIS_BLOCK = BlockHeader.parse(BytesIO(RAW_GENESIS_BLOCK))
headers = [GENESIS_BLOCK]

# connect to our peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

while len(headers) < 10000:
    # construct the message
    getheaders = GetHeadersMessage(start_block=headers[-1].hash())

    # send the message
    peer.send(getheaders)

    # wait for the "headers" response
    headers_msg = peer.wait_for(HeadersMessage)
    print(f'received {len(headers_msg.headers)} headers')
    
    # append block headers received to headers array
    for header in headers_msg.headers:
        headers.append(header)

With a bit long list of block headers, we can now validate that they actually form a chain.

The block id's that we are familiar from using block explorers aren't actually an attribute of the blocks themselves. 

![img](./genesis.png)

![img](./block.png)

Rather, they are derived from all the other fields listed above by SHA256 hashing the block twice, and interpreting this as a little-endian hexidecimal number.

So we need to implement this in order to derive `BlockHeader` id's.

For this we will need to implement three methods:
- `BlockHeader.serialize` -- which turns a `BlockHeader` into `bytes`
- `BlockHeader.hash` -- which SHA256 hashes result of `BlockHeader.serialize()` twice
- `BlockHeader.id`: which interprets `BlockHeader.hash` as hexidecimal

First, implement `BlockHeader.serialize` to get this test passing:

In [ ]:
import block as block

reload(block)
run(block.BlockHeaderTest("test_serialize"))

Next, implement `BlockHeader.hash` to get this test passing:

In [ ]:
import block as block

reload(block)
run(block.BlockHeaderTest("test_hash"))

Finally, implement `BlockHeader.id` to get this test passing:

In [ ]:
import block as block

reload(block)
run(block.BlockHeaderTest("test_id"))

Now magically the headers we downloaded earlier have id's:

In [ ]:
for header in headers[:10]:
    print(header.id())

Let's verify that they form a chain. Since `BlockHeader.prev_block` is bytes let's use `BlockHeader.hash()` to compare these directly

In [ ]:
for height, header in enumerate(headers[1:], 1):
    prev_header = headers[height-1]
    assert header.prev_block == prev_header.hash()

print('the chain is good!')

### Proof-of-Work

Now that we can validate the headers form a chain, let's validate the chain they form is ***valid***.

A few things are involved here.

First, we need to calculate the current difficulty. We take the `BlockHeader.bits` field and apply this formula: `coefficient * 256**(exponent - 3)`, where coefficient is the little-endian interpretation of the first 3 bytes of `BlockHeader.bits` and `exponent` is the last byte of `BlockHeader.bits`

Fill out the `bits_to_target` function in `lib.py` to do this. Get the following test to pass:

In [ ]:
import lib as lib

reload(lib)
run(lib.LibraryTest("test_bits_to_target"))

Now that we can calculate the target for blocks headers, see if the `nonce` provided by the blocks leads to a hash that, when interpreted as a number, is less than the target.

Update `BlockHeader.check_pow()` to get the following test to pass:

In [ ]:
import block as block

reload(block)
run(block.BlockHeaderTest("test_check_pow"))

Now let's update our code that checked whether a chain was formed to also check that proof-of-work was satisifed for each block header.

In [ ]:
for height, header in enumerate(headers[1:], 1):
    # check that a headers form a chain
    prev_header = headers[height-1]
    assert header.prev_block == prev_header.hash()

    # check that headers satisfy proof-of-work
    assert header.check_pow()

print('the chain is good!')

(also assert that `BlockHeader.bits` is always equal to the correct initial bits or something along these lines ...)

# Fetching Blocks

We now have a chain of block headers. It's time to get the full blocks -- namely, the blocks headers *plus the transactions*.

To do this, we need to send a [`getdata`](https://en.bitcoin.it/wiki/Protocol_documentation#getdata) message. This message is how you look up stuff on the bitcoin p2p network. To use it, you must construct an ["inventory vector"](https://en.bitcoin.it/wiki/Protocol_documentation#Inventory_Vectors) containing entries for each of the objects you'd like to look up. An inventory vector is simple: each item contains the id of the whatever data you're requesting (in our case block hashes) and a marker indicating what type of data this is (e.g. block or transaction).



In [ ]:
import network as network

reload(network)
run(network.GetDataMessageTest("test_serialize"))

Let's see what we get back when we send one of these:

In [ ]:
from network import GetDataMessage

# connect to peer
peer = PeerConnection(*ADDRESS, logging=True)
peer.handshake()

# send getdata
getdata = GetDataMessage()
getdata.add_block(headers[1].hash())
peer.send(getdata)

# see what we get back
while True:
    peer.read()

# Parsing Blocks

Next, we need to be able to parse full blocks.

This is involved. Blocks contain transactions, so we will need to parse transactions to pull this off.

Since I think you are beginning to get the idea we won't write all the code but we'll write some parsing code for each thing ...

In [ ]:
import tx as tx

reload(tx)
run(tx.TxTest("test_parse_version"))

In [ ]:
import script as script

reload(script)
run(script.ScriptTest("test_parse"))

In [ ]:
import tx as tx

reload(tx)
run(tx.TxTest("test_parse_inputs"))

In [ ]:
import tx as tx

reload(tx)
run(tx.TxTest("test_parse_outputs"))

In [ ]:
import tx as tx

reload(tx)
run(tx.TxTest("test_parse_locktime"))

Now we can parse blocks. Implement `Block.parse` and get this test to pass:

In [ ]:
import block as block

reload(block)
run(block.BlockTest("test_parse"))

In [ ]:
import network as network

reload(network)
run(network.BlockMessageTest("test_parse"))

In [ ]:
from network import BlockMessage

# connect to peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

# send getdata
getdata = GetDataMessage()
getdata.add_block(headers[1].hash())
peer.send(getdata)

# wait for the block message
block_msg = peer.wait_for(BlockMessage)
block_msg

In [ ]:
block = block_msg.block
block

In [ ]:
block.txns

In [ ]:
block.txns[0].tx_ins

In [ ]:
# first bitcoin ever mined ...
block.txns[0].tx_outs[0]

Now let's download the blocks corresponding to the block headers we downloaded earlier

In [ ]:
from block import Block, RAW_GENESIS_BLOCK

# Create a list of headers with genesis block filled in
GENESIS_BLOCK = Block.parse(BytesIO(RAW_GENESIS_BLOCK))
blocks = [GENESIS_BLOCK]

# connect to peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

header_index = 1

header_slice = headers[:500]

while header_index <= len(header_slice):
    # prepare and send getdata requesting 100 blocks
    getdata = GetDataMessage()
    chunk = headers[header_index:header_index+100]
    for header in chunk:
        getdata.add_block(header.hash())
    peer.send(getdata)
    
    # wait for 100 blocks to arrive, add to our list of blocks
    for _ in chunk:
        block_msg = peer.wait_for(BlockMessage)
        blocks.append(block_msg.block)
        
    header_index += len(chunk)

    print(len(blocks))

In [ ]:
# the headers match, which guarantees all other 
# BlockHeaders attributes match too

for i in range(len(header_slice)):
    assert headers[i].hash() == blocks[i].hash()

Let's make a `Blockchain` class to abstract everything we've learned so far 